In [7]:
import os
import json
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import from_json, col, to_json, struct
import watermark

%matplotlib inline
%load_ext watermark

In [8]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.7.5
IPython version      : 7.16.1

Compiler    : Clang 11.0.3 (clang-1103.0.32.62)
OS          : Darwin
Release     : 20.2.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Git hash: 4d614bd49407340479235925e61fa5afea0527f5

pandas    : 1.1.1
sys       : 3.7.5 (default, Jul  7 2020, 21:09:45) 
[Clang 11.0.3 (clang-1103.0.32.62)]
json      : 2.0.9
numpy     : 1.19.0
watermark : 2.1.0
matplotlib: 3.3.1



In [10]:
sparkSessionBuilder = SparkSession\
    .builder\
    .appName("Credit card data ingest Pipeline")

In [11]:
os.environ['PYSPARK_SUBMIT_ARGS'] = \
'--packages \
org.apache.spark:spark-streaming_2.11:2.4.5,\
org.apache.hadoop:hadoop-aws:2.7.3 \
--conf spark.jars.ivy=/tmp \
--conf spark.hadoop.fs.s3a.endpoint=http://172.30.226.86:9000 \
--conf spark.hadoop.fs.s3a.access.key=minio \
--conf spark.hadoop.fs.s3a.secret.key=minio123 \
--conf spark.hadoop.fs.s3a.path.style.access=true \
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem \
--master spark://' + os.environ['SPARK_CLUSTER'] + ':7077 pyspark-shell '


KeyError: 'SPARK_CLUSTER'

In [12]:
spark = sparkSessionBuilder.getOrCreate()
spark.sparkContext.setLogLevel("INFO")
print('Spark context started.')

Spark context started.


In [ ]:
# spark.read.csv(paths, header=False, schema="date DATE, value DECIMAL(38,4)")
dataFrame_Customer = spark.read\
                .options(delimeter=',', inferSchema='True', header='True') \
                .csv("/tmp/Customer-Churn_P1.csv")
dataFrame_Customer.printSchema()

In [13]:
dataFrame_Products = spark.read\
                .options(delimeter=',', inferSchema='True', header='True') \
                .csv("/tmp/Customer-Churn_P2.csv")
dataFrame_Products.printSchema()


AnalysisException: Path does not exist: file:/tmp/Customer-Churn_P2.csv;

In [ ]:
dataFrom_All = dataFrame_Products.join(dataFrame_Customer, "customerID", how="full")

In [ ]:
dataFrom_All.write.mode("overwrite").format("csv").save("s3a://data/full_data_csv")